# Correctly/Incorrectly Worn Face Mask Detection

## Setting up

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/

In [44]:
import sys
sys.path
sys.executable

'/Users/nguyen_l5/opt/anaconda3/envs/myenv/bin/python'

In [45]:
!which jupyter

/Users/nguyen_l5/opt/anaconda3/envs/myenv/bin/jupyter


In [46]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.layers import Input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import AveragePooling2D
from keras.models import Model, Sequential
from keras.layers import GlobalAveragePooling2D, Dropout, Dense, Activation, BatchNormalization, Conv2D, MaxPool2D, Flatten, MaxPooling2D
from tensorflow.keras.optimizers import Adam
import random
import os

In [4]:
print(keras.__version__)

2.7.0


In [5]:
keras.backend.backend()

'tensorflow'

In [6]:
print(tf.__version__)

2.7.0


In [7]:
IMAGE_SIZE = (300, 300) # resizing to reduce number of parameters
IMG_DIR = '/Users/nguyen_l5/Downloads/data'
BATCH_SIZE = 32 # small batch size to prevent Colab from crashing because of RAM shortage
NUM_CLASSES = 4

## Create dataset

In [47]:
dataset = []
for fold in os.listdir(IMG_DIR):
  if fold == 'incorrect':
    for filename in os.listdir(f'{IMG_DIR}/{fold}'):
      dataset.append((f'{fold}/{filename}', filename[11:].split('.')[0]))
  else:
    for filename in os.listdir(f'{IMG_DIR}/{fold}'):
      dataset.append((f'{fold}/{filename}', fold))

df = pd.DataFrame(dataset, columns = ['filename', 'category'])
df_train, df_test = train_test_split(df, random_state = 123, stratify = df.category, test_size = 0.2)
df_train['set'] = 'train'
df_test['set'] = 'test'
df = df_train.append(df_test)
df.to_csv('/Users/nguyen_l5/Downloads/dataset.csv', index = False)
df.head()

/Users/nguyen_l5/opt/anaconda3/envs/myenv/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/Users/nguyen_l5/opt/anaconda3/envs/myenv/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


,filename,category,set
655,correct/00118_Mask.jpg,correct,train
171,correct/00354_Mask.jpg,correct,train
223,correct/00703_Mask.jpg,correct,train
906,correct/00093_Mask.jpg,correct,train
950,incorrect/00844_Mask_Mouth_Chin.jpg,Mouth_Chin,train


In [9]:
# distribution of data among classes
df['category'].value_counts()

correct       950
Mouth_Chin    771
Nose_Mouth     81
Chin           76
Name: category, dtype: int64

In [10]:
train_df = df[df['set'] == 'train'].reset_index(drop = True)
test_df = df[df['set'] == 'test'].reset_index(drop = True)

## Traditional Neural Network

In [11]:
# creating a data generator for both the train and test set that adds noise to data

training_datagen = ImageDataGenerator(
      rescale = 1./255,
      rotation_range=5,
      horizontal_flip=True)
train_generator = training_datagen.flow_from_dataframe(
	   train_df,
     IMG_DIR,
     x_col = 'filename',
     y_col = 'category',
     color_mode = 'rgb', # traditional rgb channels
     class_mode = 'categorical',
     batch_size = BATCH_SIZE,
     seed = 123, # setting a seed to ensure consistency
     target_size = IMAGE_SIZE)

validation_datagen = ImageDataGenerator(
      rescale = 1./255)
validation_generator = validation_datagen.flow_from_dataframe(
	  test_df,
    IMG_DIR,
    x_col = 'filename',
    y_col = 'category',
    color_mode = 'rgb', # traditional rgb channels
    class_mode = 'categorical',
    batch_size = BATCH_SIZE,
	  target_size = IMAGE_SIZE,
    shuffle = False,
    seed = 123 # setting a seed to ensure consistency
    )

Found 1502 validated image filenames belonging to 4 classes.
Found 376 validated image filenames belonging to 4 classes.


In [13]:
# creating the network architecture

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation = 'relu', input_shape=IMAGE_SIZE+(3,)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (5,5), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (5,5), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Flatten(),
    
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(4, activation='softmax') # 4 because there are 4 classes
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 298, 298, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 149, 149, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 145, 145, 64)      51264     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 72, 72, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 68, 68, 128)       204928    
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 34, 34, 128)      0

2021-11-10 00:12:51.338907: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [14]:
EPOCHS = 40
INIT_LR = 1e-4 # for the Adam optimizer

# compiling the model
model.compile(
    optimizer = keras.optimizers.Adam(
        learning_rate = INIT_LR, 
        decay = INIT_LR / EPOCHS),
    loss = keras.losses.categorical_crossentropy,
    metrics = ["accuracy"],
)

In [15]:
# fitting the model

CNN = model.fit(
    train_generator, 
    epochs = EPOCHS, 
    batch_size = BATCH_SIZE,
    validation_data = validation_generator,
    validation_batch_size = BATCH_SIZE,
)

Epoch 1/40
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
17/47 [=========>....................] - ETA: 43s - loss: 1.0183 - accuracy: 0.5699

KeyboardInterrupt: 

In [ ]:
# saving the model
model.save('/gdrive/My Drive/Colab Notebooks/Mask Detection/model_weights_1020')

## Accuracy

In [48]:
new_model = tf.keras.models.load_model('/Users/nguyen_l5/Downloads/model_weights_1020')

In [49]:
new_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 298, 298, 32)      896       
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 149, 149, 32)     0         
 2D)                                                             
                                                                 
 conv2d_9 (Conv2D)           (None, 145, 145, 64)      51264     
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 72, 72, 64)       0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 68, 68, 128)       204928    
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 34, 34, 128)     

In [17]:
test_labels = test_df['category'].to_list()

In [18]:
test_df

,filename,category,set
0,correct/00358_Mask.jpg,correct,test
1,correct/00754_Mask.jpg,correct,test
2,incorrect/00501_Mask_Mouth_Chin.jpg,Mouth_Chin,test
3,correct/00607_Mask.jpg,correct,test
4,incorrect/00861_Mask_Mouth_Chin.jpg,Mouth_Chin,test
...,...,...,...
371,correct/00138_Mask.jpg,correct,test
372,incorrect/00448_Mask_Chin.jpg,Chin,test
373,correct/00034_Mask.jpg,correct,test
374,correct/00217_Mask.jpg,correct,test


In [21]:
validation_classes = []
validation_images = []

for i in range( -(-validation_generator.samples // validation_generator.batch_size)):
   batch = validation_generator.next()
   expected = np.argmax(batch[1], axis = 1) 
   validation_classes.extend(expected)
   validation_images.extend(batch[0])

In [25]:
validation_generator.samples

376

In [28]:
validation_generator.next()

(array([[[[0.18039216, 0.11764707, 0.05882353],
          [0.20784315, 0.13333334, 0.07843138],
          [0.21568629, 0.12941177, 0.07450981],
          ...,
          [0.03529412, 0.03921569, 0.01960784],
          [0.03529412, 0.03137255, 0.01568628],
          [0.04313726, 0.02745098, 0.01568628]],
 
         [[0.20784315, 0.13333334, 0.07450981],
          [0.22352943, 0.14901961, 0.09411766],
          [0.20784315, 0.13333334, 0.07843138],
          ...,
          [0.03137255, 0.03529412, 0.01176471],
          [0.03137255, 0.03529412, 0.01176471],
          [0.03529412, 0.03137255, 0.01176471]],
 
         [[0.22352943, 0.14117648, 0.07450981],
          [0.21960786, 0.14509805, 0.08627451],
          [0.20392159, 0.12941177, 0.07058824],
          ...,
          [0.04313726, 0.03921569, 0.01960784],
          [0.04313726, 0.02745098, 0.01568628],
          [0.04705883, 0.03529412, 0.01568628]],
 
         ...,
 
         [[0.5372549 , 0.6039216 , 0.6745098 ],
          [0.54901

In [27]:
test_classes = []
test_images = []

for i in range(len(test_df)):
    expected = np.argmax()

11

In [96]:
from keras.preprocessing import image
pred_dir = '/Users/nguyen_l5/Downloads/test.jpg'
img = image.load_img(pred_dir, target_size=(300, 300))
img_tensor = image.img_to_array(img)
img_tensor = np.expand_dims(img_tensor, axis=0)
img_tensor /= 255.

In [97]:
print(new_model.predict(img_tensor).argmax(axis=-1))

[0]


In [61]:
# Evaluate the restored model
loss, acc = new_model.evaluate(test_df, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

print(new_model.predict(test_df).shape)

ValueError: Failed to find data adapter that can handle input: <class 'pandas.core.frame.DataFrame'>, (<class 'list'> containing values of types {"<class 'str'>"})

In [19]:
def show_accuracy(history):
  acc = history.history['accuracy']
  val_acc = history.history['val_accuracy']
  loss = history.history['loss']
  val_loss = history.history['val_loss']
  epochs = range(1, len(acc)+1)
  print("acc: ", acc)
  print("val_acc: ", val_acc)
  print("loss: ", loss)
  print("val_loss: ", val_loss)
  
  fig, ax1 = plt.subplots()
  fig.tight_layout()
  ax1.axis(ymin = 0.3, ymax = 1.0)
  ax1.set_xlabel('epocs')
  ax1.set_ylabel('accurary')
  ax1.plot(epochs, acc, 'r--', label='Training accuracy')
  ax1.plot(epochs, val_acc, 'b', label='Validation accuracy')
  ax2 = ax1.twinx() 
  ax2.set_ylabel('loss')
  ax2.axis(ymin = 0.0, ymax = 2.7)
  ax2.plot(epochs, loss, 'r--', label='Training')
  ax2.plot(epochs, val_loss, 'b', label='Validation')
  plt.title('Training and validation accuracy / loss')
  plt.legend(loc="lower left")#"best")#0
  plt.show()

show_accuracy(new_model)

TypeError: 'NoneType' object is not subscriptable

In [ ]:
from collections import Counter
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import itertools

def plot_confusion_matrix(cm, classes_map,
                          normalize=False,
                          title='Confusion Matrix',
                          cmap=plt.cm.Blues):

    plt.imshow(cm, interpolation='nearest', cmap = cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes_map))
    inv_map = {v: k for k, v in classes_map.items()}
    labels = inv_map.values()
    plt.xticks(tick_marks, labels, rotation=45)
    plt.yticks(tick_marks, labels)
    if normalize:
        cm = np.around(cm.astype('float') / cm.sum(axis = 1)[:, np.newaxis], 2)
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color = "white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True class')
    plt.xlabel('Predicted class')


validation_classes = []
validation_images = []

for i in range( -(-validation_generator.samples // validation_generator.batch_size)):
   batch = validation_generator.next()
   expected = np.argmax(batch[1], axis = 1) 
   validation_classes.extend(expected)
   validation_images.extend(batch[0])

validation_classes = np.array(validation_classes)
validation_images = np.array(validation_images)

Y_pred = model.predict(validation_images)
y_pred = np.argmax(Y_pred, axis=1)


print(classification_report(validation_classes, y_pred, 
		target_names = ['CHIN', 'MOUTH_CHIN', 'NOSE_MOUTH', 'CORRECT']))

cfs_mt = confusion_matrix(validation_classes, y_pred)
classes = {'CHIN': 0, 'MOUTH_CHIN': 1, 'NOSE_MOUTH': 2, 'CORRECT': 3}
plot_confusion_matrix(cfs_mt, classes_map = classes)